In [68]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
import statsmodels.stats.multitest as smm

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [3]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [4]:
data_n = data[data.Diagnosis=='normal']
data_en = data[data.Diagnosis=='early neoplasia']
data_c = data[data.Diagnosis=='cancer']

In [5]:
p_n_en = []
p_en_c = []
for k in range(2, data.shape[1], 1):
    p_n_en.append(scipy.stats.ttest_ind(data_n.iloc[:,k].values, data_en.iloc[:,k].values, equal_var = False)[1])
    p_en_c.append(scipy.stats.ttest_ind(data_en.iloc[:,k].values, data_c.iloc[:,k].values, equal_var = False)[1])

In [6]:
i1 = 0
i2 = 0

for p in p_n_en:
    if p<=0.05:
        i1+=1
for p in p_en_c:
    if p<=0.05:
        i2+=1

In [7]:
f1 = open('1_1.txt', 'w')
f2 = open('1_2.txt', 'w')

f1.write('%i'% i1)
f2.write('%i'% i2)

f1.close()
f2.close()

In [73]:
rejectnen, pnen_corrected, a1nen, a2nen = smm.multipletests(p_n_en, alpha=0.0025, method='holm')

rejectenc, penc_corrected, a1enc, a2enc = smm.multipletests(p_en_c, alpha=0.0025, method='holm')

In [74]:
print data_n.shape, data_en.shape, data_c.shape

(24, 15750) (25, 15750) (23, 15750)


In [79]:
fc_nen = []
fc_enc = []

for k in range(2, len(pnen_corrected)+2, 1):
    if pnen_corrected[k-2] < 0.025:
        if data_en.iloc[:, k].mean() > data_n.iloc[:, k].mean():
            fc_nen.append(data_en.iloc[:, k].mean()/data_n.iloc[:, k].mean())
        else:
            fc_nen.append(-data_n.iloc[:, k].mean()/data_en.iloc[:, k].mean())
            
for k in range(2, len(penc_corrected)+2, 1):
    if penc_corrected[k-2] < 0.025:
        if data_c.iloc[:, k].mean() > data_en.iloc[:, k].mean():
            fc_enc.append(data_c.iloc[:, k].mean()/data_en.iloc[:, k].mean())
        else:
            fc_enc.append(-data_en.iloc[:, k].mean()/data_c.iloc[:, k].mean())

In [80]:
def ans(X):
    a = 0
    for x in X:
        if abs(x)>1.5:
            a+=1
    return a

In [81]:
f1 = open('2_1.txt', 'w')
f2 = open('2_2.txt', 'w')

f1.write('%i' % ans(fc_nen))
f2.write('%i' % ans(fc_enc))

f1.close()
f2.close()

In [83]:
rejectnen, pnen_corrected, a1nen, a2nen = smm.multipletests(p_n_en, alpha=0.05, method='fdr_bh')

rejectenc, penc_corrected, a1enc, a2enc = smm.multipletests(p_en_c, alpha=0.05, method='fdr_bh')

In [84]:
fc_nen = []
fc_enc = []

for k in range(2, len(pnen_corrected)+2, 1):
    if pnen_corrected[k-2] < 0.025:
        if data_en.iloc[:, k].mean() > data_n.iloc[:, k].mean():
            fc_nen.append(data_en.iloc[:, k].mean()/data_n.iloc[:, k].mean())
        else:
            fc_nen.append(-data_n.iloc[:, k].mean()/data_en.iloc[:, k].mean())
            
for k in range(2, len(penc_corrected)+2, 1):
    if penc_corrected[k-2] < 0.025:
        if data_c.iloc[:, k].mean() > data_en.iloc[:, k].mean():
            fc_enc.append(data_c.iloc[:, k].mean()/data_en.iloc[:, k].mean())
        else:
            fc_enc.append(-data_en.iloc[:, k].mean()/data_c.iloc[:, k].mean())

In [85]:
f1 = open('3_1.txt', 'w')
f2 = open('3_2.txt', 'w')

f1.write('%i' % ans(fc_nen))
f2.write('%i' % ans(fc_enc))

f1.close()
f2.close()